In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
from pandas.api.types import is_numeric_dtype
from functools import reduce
from pandas.api.types import is_numeric_dtype
from itertools import groupby
from collections import OrderedDict

In [2]:
df = pd.read_csv('../data/Echo/echo-40-attcon.csv')

In [3]:
appr_df = df.copy()

In [4]:
df

,Age,Pericardial,Fractional,EPSS,LVDD,Wall-score,Wall-index,Alive
0,71,0,-,9,4.6,14,-,0
1,72,0,0.38,6,4.1,-,1.7,0
2,55,0,0.26,4,-,-,-,0
3,60,-,0.253,12.062,-,-,-,0
4,57,-,0.16,-,5.75,-,-,0
5,-,-,0.26,-,4.31,12,1,0
6,62,0,0.23,-,5.43,22.5,1.875,0
7,60,0,0.33,8,5.25,14,1,0
8,46,0,0.34,0,-,16,1.14,0
9,54,0,0.14,13,4.49,15.5,-,0


In [5]:
df_headers = list(df)
concept = df_headers[-1]
concept

'Alive'

In [6]:
#all unique concepts
concept_list = df[concept].unique()
concept_list

array([0, 1])

In [7]:
#calculating cases by concepts and making sets
U = [] #universal list containing all cases
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        U.append(index+1)
        if row[concept] == item:
            temp_list.append(index)
    goal_list.append(temp_list)
    temp_list = []

In [8]:
print(goal_list,)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71], [10, 14, 16, 17, 19, 20, 21, 22, 23, 28, 36, 37, 38, 40, 51, 56, 59, 60, 63, 65, 66, 69, 72, 73]]


In [9]:
attributes = list(df)
attributes

['Age',
 'Pericardial',
 'Fractional',
 'EPSS',
 'LVDD',
 'Wall-score',
 'Wall-index',
 'Alive']

In [10]:
case_list = []

In [11]:
def discretize(numeric_col):
    #Sorting the values of numeric column
    sort_col = df[numeric_col].sort_values()
   
    df['sort_col'] = sort_col.values
    point_list = df['sort_col'].unique()
    point_list = list(point_list)
    if '-' in point_list:
        point_list.remove('-')
    
    #Finding average between each two points
    avg_list = []
    for i in range(len(point_list)-1):
        avg = (float(point_list[i]) + float(point_list[i+1]))/2
        avg_list.append(round(float(avg),2))
        
    #Performing the discretization and adding the cases
    for i in avg_list:
        case = str(numeric_col) + "," + str(round(float(point_list[0]),1)) + ".." + str(i)
        case2 = str(numeric_col) + "," + str(i) + ".." + str(round(float(point_list[len(point_list)-1]),1))
        case_list.append(case)
        case_list.append(case2)

In [12]:
#Discretization considering upto 2 decimal point
for item in attributes[:-1]:
        discretize(item)

In [13]:
case_list = list(OrderedDict.fromkeys(case_list))

In [14]:
case_list

['Age,46.0..48.5',
 'Age,48.5..85.0',
 'Age,46.0..52.5',
 'Age,52.5..85.0',
 'Age,46.0..54.5',
 'Age,54.5..85.0',
 'Age,46.0..55.5',
 'Age,55.5..85.0',
 'Age,46.0..56.5',
 'Age,56.5..85.0',
 'Age,46.0..57.5',
 'Age,57.5..85.0',
 'Age,46.0..58.5',
 'Age,58.5..85.0',
 'Age,46.0..59.5',
 'Age,59.5..85.0',
 'Age,46.0..60.5',
 'Age,60.5..85.0',
 'Age,46.0..61.5',
 'Age,61.5..85.0',
 'Age,46.0..62.5',
 'Age,62.5..85.0',
 'Age,46.0..64.0',
 'Age,64.0..85.0',
 'Age,46.0..65.5',
 'Age,65.5..85.0',
 'Age,46.0..66.73',
 'Age,66.73..85.0',
 'Age,46.0..68.23',
 'Age,68.23..85.0',
 'Age,46.0..69.5',
 'Age,69.5..85.0',
 'Age,46.0..70.5',
 'Age,70.5..85.0',
 'Age,46.0..71.5',
 'Age,71.5..85.0',
 'Age,46.0..72.5',
 'Age,72.5..85.0',
 'Age,46.0..73.5',
 'Age,73.5..85.0',
 'Age,46.0..75.5',
 'Age,75.5..85.0',
 'Age,46.0..78.0',
 'Age,78.0..85.0',
 'Age,46.0..82.0',
 'Age,82.0..85.0',
 'Pericardial,0.0..0.5',
 'Pericardial,0.5..1.0',
 'Fractional,0.1..0.07',
 'Fractional,0.07..0.5',
 'Fractional,0.1..0.08

In [15]:
df2 = df.copy()

In [16]:
for col in df:
    for i, row_value in df[col].iteritems():
        
        #This is for attribute-concept value interpretation
        #For Lost value interpretation, we don't add that case to any block at all
        if row_value == '-':
            
            final_values = []
            print(col,i)
            concept_tmp = df[concept][i]
            print(concept_tmp)
            
            for item_list in goal_list:
                if i in item_list:
                    final_values = item_list
                    
            print (final_values)
            
            #Taking out all unique cases corresponding to the matched concept
            col_values = []
            for item in final_values:
                col_values.append(df[col][item])
        
            
            uniqueList = []
            for letter in col_values:
                if letter not in uniqueList:
                    uniqueList.append(letter)
            
            if '-' in uniqueList:
                uniqueList.remove('-')
                
            if '?' in uniqueList:
                uniqueList.remove('?')
            print(uniqueList)
            
#             print(df.loc[i,col])
            df2.at[i, col] = uniqueList
            
    print("\n")

Age 5
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71]
['71', '72', '55', '60', '57', '62', '46', '54', '73', '63', '61', '79', '58', '51', '70', '56', '59', '74', '66']
Age 13
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71]
['71', '72', '55', '60', '57', '62', '46', '54', '73', '63', '61', '79', '58', '51', '70', '56', '59', '74', '66']
Age 14
1
[10, 14, 16, 17, 19, 20, 21, 22, 23, 28, 36, 37, 38, 40, 51, 56, 59, 60, 63, 65, 66, 69, 72, 73]
['77', '66', '69', '85', '73', '71', '61', '60', '67.4578', '65', '63', '62']
Age 16
1
[10, 14, 16, 17, 19, 20, 21, 22, 23, 28, 36, 37, 38, 40, 51, 56, 59, 60, 63, 65, 66, 69, 72, 73]
['77', '66', '69', '85', '73', '71', '61', '60', '67.4578', '65', '63', '62']
Age

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71]
['9', '6', '4', '12.062', '8', '0', '13', '10', '4.2', '13.1', '11.0272', '9.3', '21.3', '9.4', '18.6', '11.9', '12', '8.8', '11', '7.1', '16.8', '9.1', '9.7']
EPSS 11
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71]
['9', '6', '4', '12.062', '8', '0', '13', '10', '4.2', '13.1', '11.0272', '9.3', '21.3', '9.4', '18.6', '11.9', '12', '8.8', '11', '7.1', '16.8', '9.1', '9.7']
EPSS 15
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71]
['9', '6', '4', '12.062', '8', '0', '13', '10', '4.2', '13.1', '11.0272', '9.3', '21.3'

['14', '12', '22.5', '16', '15.5', '13', '11', '22', '9', '17', '10.5', '5.5', '13.67', '15', '16.5', '18', '12.5', '10', '13.5']
Wall-score 44
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71]
['14', '12', '22.5', '16', '15.5', '13', '11', '22', '9', '17', '10.5', '5.5', '13.67', '15', '16.5', '18', '12.5', '10', '13.5']
Wall-score 47
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71]
['14', '12', '22.5', '16', '15.5', '13', '11', '22', '9', '17', '10.5', '5.5', '13.67', '15', '16.5', '18', '12.5', '10', '13.5']
Wall-score 51
1
[10, 14, 16, 17, 19, 20, 21, 22, 23, 28, 36, 37, 38, 40, 51, 56, 59, 60, 63, 65, 66, 69, 72, 73]
['18', '11.67', '24', '27', '19.5', '7.5', '8', '21.5', '23', '18.7787', '39', '28'

In [17]:
df2

,Age,Pericardial,Fractional,EPSS,LVDD,Wall-score,Wall-index,Alive,sort_col
0,71,0,"[0.38, 0.26, 0.253, 0.16, 0.23, 0.33, 0.34, 0....",9,4.6,14,"[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
1,72,0,0.38,6,4.1,"[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...",1.7,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
2,55,0,0.26,4,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....","[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
3,60,"[0, 1]",0.253,12.062,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....","[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
4,57,"[0, 1]",0.16,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",5.75,"[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
5,"[71, 72, 55, 60, 57, 62, 46, 54, 73, 63, 61, 7...","[0, 1]",0.26,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",4.31,12,1,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
6,62,0,0.23,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",5.43,22.5,1.875,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
7,60,0,0.33,8,5.25,14,1,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
8,46,0,0.34,0,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....",16,1.14,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
9,54,0,0.14,13,4.49,15.5,"[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."


In [18]:
appr_df_attcon = df2.copy()

In [19]:
df = df2.copy()

In [20]:
df_headers = list(df)
concept = df_headers[-2]
concept_list = df[concept].unique()

In [21]:
concept_list

array([0, 1])

In [22]:
#calculating cases by concepts and making sets
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        if row[concept] == item:
            temp_list.append(index+1)
    goal_list.append(temp_list)
    temp_list = []

In [23]:
print(goal_list,)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 58, 59, 62, 63, 65, 68, 69, 71, 72], [11, 15, 17, 18, 20, 21, 22, 23, 24, 29, 37, 38, 39, 41, 52, 57, 60, 61, 64, 66, 67, 70, 73, 74]]


In [24]:
df

,Age,Pericardial,Fractional,EPSS,LVDD,Wall-score,Wall-index,Alive,sort_col
0,71,0,"[0.38, 0.26, 0.253, 0.16, 0.23, 0.33, 0.34, 0....",9,4.6,14,"[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
1,72,0,0.38,6,4.1,"[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...",1.7,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
2,55,0,0.26,4,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....","[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
3,60,"[0, 1]",0.253,12.062,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....","[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
4,57,"[0, 1]",0.16,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",5.75,"[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
5,"[71, 72, 55, 60, 57, 62, 46, 54, 73, 63, 61, 7...","[0, 1]",0.26,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",4.31,12,1,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
6,62,0,0.23,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",5.43,22.5,1.875,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
7,60,0,0.33,8,5.25,14,1,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
8,46,0,0.34,0,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....",16,1.14,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
9,54,0,0.14,13,4.49,15.5,"[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."


In [25]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) >= float(start) and float(case) <= float(end):
                        temp_list.append(index+1)
                   
                
            else:
                if float(row[a]) >= float(start) and float(row[a]) <= float(end):
                    temp_list.append(index+1)
                
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) == b:
                        temp_list.append(index+1)
            else:           
                if float(row[a]) == b:
                    temp_list.append(index+1)
        
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []

In [26]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       Age,46.0..48.5      [32, 34, 35, 36, 6, 9, 42, 43, 44, 45, 14, 50, 19, 51, 58, 59, 28, 30, 31]
1       Age,48.5..85.0      [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
2       Age,46.0..52.5      [32, 34, 35, 36, 6, 9, 42, 43, 44, 45, 14, 48, 50, 19, 51, 58, 59, 28, 30, 31]
3       Age,52.5..85.0      [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
4       Age,46.0..54.5      [6, 9, 10, 14, 19, 28, 30, 31, 32, 34, 35, 36, 42, 43, 44, 45, 48, 50, 51, 56, 58, 59]
5       Age,54.5..85.0      [1, 2, 3, 4, 5, 6, 7, 8, 

In [27]:
#Creating data for case and att-value list
data = {'Cases': case_list, 'att_val': att_val_list}

In [28]:
df2 = pd.DataFrame(data)

In [29]:
#Cases and corresponding att-value pairs
df2

,Cases,att_val
0,"Age,46.0..48.5","[32, 34, 35, 36, 6, 9, 42, 43, 44, 45, 14, 50,..."
1,"Age,48.5..85.0","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1..."
2,"Age,46.0..52.5","[32, 34, 35, 36, 6, 9, 42, 43, 44, 45, 14, 48,..."
3,"Age,52.5..85.0","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1..."
4,"Age,46.0..54.5","[6, 9, 10, 14, 19, 28, 30, 31, 32, 34, 35, 36,..."
5,"Age,54.5..85.0","[1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 1..."
6,"Age,46.0..55.5","[3, 6, 9, 10, 14, 16, 19, 25, 28, 30, 31, 32, ..."
7,"Age,55.5..85.0","[1, 2, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 17, ..."
8,"Age,46.0..56.5","[3, 6, 9, 10, 14, 16, 19, 25, 28, 30, 31, 32, ..."
9,"Age,56.5..85.0","[1, 2, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 17, ..."


### Approximations !!

In [30]:
attributes = list(appr_df)
del attributes[-1]

In [31]:
attributes

['Age',
 'Pericardial',
 'Fractional',
 'EPSS',
 'LVDD',
 'Wall-score',
 'Wall-index']

In [32]:
appr_df

,Age,Pericardial,Fractional,EPSS,LVDD,Wall-score,Wall-index,Alive
0,71,0,-,9,4.6,14,-,0
1,72,0,0.38,6,4.1,-,1.7,0
2,55,0,0.26,4,-,-,-,0
3,60,-,0.253,12.062,-,-,-,0
4,57,-,0.16,-,5.75,-,-,0
5,-,-,0.26,-,4.31,12,1,0
6,62,0,0.23,-,5.43,22.5,1.875,0
7,60,0,0.33,8,5.25,14,1,0
8,46,0,0.34,0,-,16,1.14,0
9,54,0,0.14,13,4.49,15.5,-,0


In [33]:
case_list = []
#Loop through all the attributes except last 2 - Concept and sort_col
for item in attributes:
    print(item)
    
    #check for non numeric columns
    if not is_numeric_dtype(appr_df[item]):
        temp = appr_df[item].unique()
        print(temp)
        for i in temp:
            if i == '?' or i == '-':
                continue
            else:
                case = item + "," + i
                case_list.append(case)

Age
['71' '72' '55' '60' '57' '-' '62' '46' '54' '77' '73' '66' '69' '85' '63'
 '61' '79' '58' '51' '70' '56' '59' '67.4578' '74' '65']
Pericardial
['0' '-' '1']
Fractional
['-' '0.38' '0.26' '0.253' '0.16' '0.23' '0.33' '0.34' '0.14' '0.13'
 '0.45' '0.15' '0.12' '0.25' '0.09' '0.22' '0.21' '0.241021' '0.06' '0.51'
 '0.35' '0.171136' '0.3' '0.17' '0.187' '0.2' '0.07']
EPSS
['9' '6' '4' '12.062' '-' '8' '0' '13' '16' '10' '23' '11' '20' '19'
 '12.733' '4.2' '13.1' '11.0272' '23.6' '9.3' '21.3' '15.8968' '9.4'
 '24.6' '18.6' '11.9' '12' '8.8' '19.4' '7.1' '16.8' '9.1' '9.7' '40'
 '7.6']
LVDD
['4.6' '4.1' '-' '5.75' '4.31' '5.43' '5.25' '4.49' '4' '3.73' '5.8'
 '4.29' '5.2' '5.819' '5.4' '5.39' '6.06' '4.07' '5.31' '5.1145' '3.88'
 '3.63' '3.59' '4.649' '6.29' '3.49' '5.65' '4.57' '5.3' '5.15' '5.02'
 '5.26' '5.78' '4.72' '4.54' '4.58' '4.44' '4.56' '5.47' '6.73' '4.55'
 '3.52' '5.49' '4.42']
Wall-score
['14' '-' '12' '22.5' '16' '15.5' '18' '11.67' '24' '27' '19.5' '7.5' '8'
 '13' '21.5'

In [34]:
case_list

['Age,71',
 'Age,72',
 'Age,55',
 'Age,60',
 'Age,57',
 'Age,62',
 'Age,46',
 'Age,54',
 'Age,77',
 'Age,73',
 'Age,66',
 'Age,69',
 'Age,85',
 'Age,63',
 'Age,61',
 'Age,79',
 'Age,58',
 'Age,51',
 'Age,70',
 'Age,56',
 'Age,59',
 'Age,67.4578',
 'Age,74',
 'Age,65',
 'Pericardial,0',
 'Pericardial,1',
 'Fractional,0.38',
 'Fractional,0.26',
 'Fractional,0.253',
 'Fractional,0.16',
 'Fractional,0.23',
 'Fractional,0.33',
 'Fractional,0.34',
 'Fractional,0.14',
 'Fractional,0.13',
 'Fractional,0.45',
 'Fractional,0.15',
 'Fractional,0.12',
 'Fractional,0.25',
 'Fractional,0.09',
 'Fractional,0.22',
 'Fractional,0.21',
 'Fractional,0.241021',
 'Fractional,0.06',
 'Fractional,0.51',
 'Fractional,0.35',
 'Fractional,0.171136',
 'Fractional,0.3',
 'Fractional,0.17',
 'Fractional,0.187',
 'Fractional,0.2',
 'Fractional,0.07',
 'EPSS,9',
 'EPSS,6',
 'EPSS,4',
 'EPSS,12.062',
 'EPSS,8',
 'EPSS,0',
 'EPSS,13',
 'EPSS,16',
 'EPSS,10',
 'EPSS,23',
 'EPSS,11',
 'EPSS,20',
 'EPSS,19',
 'EPSS,12.73

In [35]:
appr_df_attcon

,Age,Pericardial,Fractional,EPSS,LVDD,Wall-score,Wall-index,Alive,sort_col
0,71,0,"[0.38, 0.26, 0.253, 0.16, 0.23, 0.33, 0.34, 0....",9,4.6,14,"[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
1,72,0,0.38,6,4.1,"[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...",1.7,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
2,55,0,0.26,4,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....","[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
3,60,"[0, 1]",0.253,12.062,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....","[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
4,57,"[0, 1]",0.16,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",5.75,"[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
5,"[71, 72, 55, 60, 57, 62, 46, 54, 73, 63, 61, 7...","[0, 1]",0.26,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",4.31,12,1,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
6,62,0,0.23,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",5.43,22.5,1.875,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
7,60,0,0.33,8,5.25,14,1,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
8,46,0,0.34,0,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....",16,1.14,0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."
9,54,0,0.14,13,4.49,15.5,"[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0,"[1, 1.04, 1.08, 1.1, 1.14, 1.18, 1.222, 1.25, ..."


In [36]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in appr_df_attcon.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) >= float(start) and float(case) <= float(end):
                        temp_list.append(index+1)
                   
                
            else:
                if float(row[a]) >= float(start) and float(row[a]) <= float(end):
                    temp_list.append(index+1)
                
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in appr_df_attcon.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) == float(b):
                        temp_list.append(index+1)
            else:           
                if float(row[a]) == float(b):
                    temp_list.append(index+1)
        
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []

In [37]:
df = df.drop(columns=['sort_col'])

In [38]:
df

,Age,Pericardial,Fractional,EPSS,LVDD,Wall-score,Wall-index,Alive
0,71,0,"[0.38, 0.26, 0.253, 0.16, 0.23, 0.33, 0.34, 0....",9,4.6,14,"[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0
1,72,0,0.38,6,4.1,"[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...",1.7,0
2,55,0,0.26,4,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....","[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0
3,60,"[0, 1]",0.253,12.062,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....","[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0
4,57,"[0, 1]",0.16,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",5.75,"[14, 12, 22.5, 16, 15.5, 13, 11, 22, 9, 17, 10...","[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0
5,"[71, 72, 55, 60, 57, 62, 46, 54, 73, 63, 61, 7...","[0, 1]",0.26,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",4.31,12,1,0
6,62,0,0.23,"[9, 6, 4, 12.062, 8, 0, 13, 10, 4.2, 13.1, 11....",5.43,22.5,1.875,0
7,60,0,0.33,8,5.25,14,1,0
8,46,0,0.34,0,"[4.6, 4.1, 5.75, 4.31, 5.43, 5.25, 4.49, 4, 3....",16,1.14,0
9,54,0,0.14,13,4.49,15.5,"[1.7, 1, 1.875, 1.14, 1.56, 1.625, 1.222, 2, 1...",0


In [39]:
U = set(U)
print(U,)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}


In [40]:
# Creating dictionary combining case_list and att_val list
block = dict(zip(case_list, att_val_list))
block

{'Age,71': [1,
  6,
  14,
  15,
  17,
  18,
  19,
  24,
  28,
  29,
  30,
  31,
  32,
  34,
  35,
  36,
  37,
  38,
  42,
  43,
  44,
  45,
  50,
  51,
  52,
  58,
  59,
  60,
  61,
  64,
  67,
  73],
 'Age,72': [32,
  2,
  34,
  35,
  36,
  6,
  42,
  43,
  44,
  45,
  14,
  47,
  50,
  19,
  51,
  58,
  59,
  28,
  30,
  31],
 'Age,55': [3,
  6,
  14,
  16,
  19,
  25,
  28,
  30,
  31,
  32,
  34,
  35,
  36,
  42,
  43,
  44,
  45,
  50,
  51,
  58,
  59,
  62],
 'Age,60': [4,
  6,
  8,
  14,
  15,
  17,
  18,
  19,
  28,
  29,
  30,
  31,
  32,
  34,
  35,
  36,
  37,
  38,
  41,
  42,
  43,
  44,
  45,
  50,
  51,
  52,
  54,
  58,
  59,
  60,
  61,
  64,
  67,
  73],
 'Age,57': [32,
  34,
  35,
  36,
  5,
  6,
  72,
  42,
  43,
  44,
  45,
  14,
  50,
  19,
  51,
  58,
  59,
  28,
  30,
  31],
 'Age,62': [6,
  7,
  12,
  14,
  15,
  17,
  18,
  19,
  28,
  29,
  30,
  31,
  32,
  34,
  35,
  36,
  37,
  38,
  42,
  43,
  44,
  45,
  50,
  51,
  52,
  58,
  59,
  60,
  61,
  64,


In [41]:
attributes = list(df)
attributes

['Age',
 'Pericardial',
 'Fractional',
 'EPSS',
 'LVDD',
 'Wall-score',
 'Wall-index',
 'Alive']

In [42]:
dic = {}
for index, row in df.iterrows():
    tmp_set = set()
    final_union = []
    char_list = []
    char_list_2 = []
    final_union_set = []
  
    for cols in attributes[:-1]:
        #If the value for corresponding attribute is a list then create all of the att-value pairs
        if type(df.loc[index,cols]) == list:
            print("When values are list") #########
            for item in df.loc[index,cols]:
                block_key = cols + "," + item
                char_list.append(block_key) #char_list has all att-val cases
                print(char_list) #########
                
            union_set = set()
            #Compute union of att-concept value case
            for item in char_list:
                union_set = union_set.union(set(block[item]))
            
            print("Union Set: ", union_set) #########
            final_union.append(union_set)
        
        else:
            print("When value is single") #########
            block_key = cols + "," + df.loc[index,cols]
            char_list_2.append(block_key) #char_list_2 has all single cases
            print(char_list_2) ##########
   
    #Compute instersection for this current row for Characteristics set
    
    print("final_union: ", final_union) ########
    if len(final_union):
        final_union_set = list(reduce(set.intersection, [set(item) for item in final_union]))
        
    
    print("Final Union Set: ", final_union_set) ##########
    
    for item in char_list_2:
        if item in block:
            print("When item is found as key in the block") ########
            if tmp_set == set():
                #Copy over the current set elements to B
                for i in range(len(block[item])):
                    tmp_set.add(block[item][i])
                    
            tmp_set = tmp_set.intersection(set(block[item]))
            print(tmp_set) ########
        
        #If item not in block
        else:
            print("When item is not found as key in the block") ########
            print(tmp_set) ########
            print(U) #########
            if tmp_set == set():
                tmp_set = U
                
            tmp_set = tmp_set.intersection(U)
            print(tmp_set) ########
    
    print("Final tmp_set: ", tmp_set) #########
    final_union_set = set(final_union_set)
    
    if final_union_set == set():
        tmp_set = tmp_set
    else:
        tmp_set = tmp_set.intersection(final_union_set)
        
    print("This is the final value: ", tmp_set) ###########
        
    key = ('K_%d' % (index+1))
    print(key) #########
    dic[key] = tmp_set
    print("\n") ##########

When value is single
['Age,71']
When value is single
['Age,71', 'Pericardial,0']
When values are list
['Fractional,0.38']
['Fractional,0.38', 'Fractional,0.26']
['Fractional,0.38', 'Fractional,0.26', 'Fractional,0.253']
['Fractional,0.38', 'Fractional,0.26', 'Fractional,0.253', 'Fractional,0.16']
['Fractional,0.38', 'Fractional,0.26', 'Fractional,0.253', 'Fractional,0.16', 'Fractional,0.23']
['Fractional,0.38', 'Fractional,0.26', 'Fractional,0.253', 'Fractional,0.16', 'Fractional,0.23', 'Fractional,0.33']
['Fractional,0.38', 'Fractional,0.26', 'Fractional,0.253', 'Fractional,0.16', 'Fractional,0.23', 'Fractional,0.33', 'Fractional,0.34']
['Fractional,0.38', 'Fractional,0.26', 'Fractional,0.253', 'Fractional,0.16', 'Fractional,0.23', 'Fractional,0.33', 'Fractional,0.34', 'Fractional,0.14']
['Fractional,0.38', 'Fractional,0.26', 'Fractional,0.253', 'Fractional,0.16', 'Fractional,0.23', 'Fractional,0.33', 'Fractional,0.34', 'Fractional,0.14', 'Fractional,0.45']
['Fractional,0.38', 'Fracti

['Age,77', 'Age,66', 'Age,69', 'Age,85', 'Age,73', 'Age,71', 'Age,61', 'Age,60', 'Age,67.4578', 'Age,65', 'Age,63', 'Age,62', 'LVDD,5.8', 'LVDD,5.2', 'LVDD,5.4', 'LVDD,5.39', 'LVDD,6.06', 'LVDD,5.1145', 'LVDD,5.65', 'LVDD,4.44', 'LVDD,5.47', 'LVDD,6.73', 'LVDD,3.52', 'LVDD,4.42']
Union Set:  {1, 4, 6, 7, 8, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 50, 51, 52, 54, 57, 58, 59, 60, 61, 64, 66, 67, 68, 70, 73, 74}
When value is single
['Pericardial,1', 'Fractional,0.26', 'EPSS,11', 'Wall-score,18']
When values are list
['Age,77', 'Age,66', 'Age,69', 'Age,85', 'Age,73', 'Age,71', 'Age,61', 'Age,60', 'Age,67.4578', 'Age,65', 'Age,63', 'Age,62', 'LVDD,5.8', 'LVDD,5.2', 'LVDD,5.4', 'LVDD,5.39', 'LVDD,6.06', 'LVDD,5.1145', 'LVDD,5.65', 'LVDD,4.44', 'LVDD,5.47', 'LVDD,6.73', 'LVDD,3.52', 'LVDD,4.42', 'Wall-index,2.33']
['Age,77', 'Age,66', 'Age,69', 'Age,85', 'Age,73', 'Age,71', 'Age,61', 'Age,60', 'Age,67.4578', 

When value is single
['Age,71', 'Wall-score,8']
When values are list
['Pericardial,0', 'Pericardial,1', 'Fractional,0.13', 'Fractional,0.12', 'Fractional,0.26', 'Fractional,0.22', 'Fractional,0.23', 'Fractional,0.06', 'Fractional,0.171136', 'Fractional,0.17', 'Fractional,0.3', 'EPSS,16', 'EPSS,23', 'EPSS,11', 'EPSS,20', 'EPSS,19', 'EPSS,12.733', 'EPSS,23.6', 'EPSS,15.8968', 'EPSS,24.6', 'EPSS,19.4', 'EPSS,9.4', 'EPSS,40', 'EPSS,7.6', 'LVDD,5.8', 'LVDD,5.2', 'LVDD,5.4', 'LVDD,5.39', 'LVDD,6.06', 'LVDD,5.1145', 'LVDD,5.65', 'LVDD,4.44', 'LVDD,5.47', 'LVDD,6.73', 'LVDD,3.52', 'LVDD,4.42', 'Wall-index,2.33']
['Pericardial,0', 'Pericardial,1', 'Fractional,0.13', 'Fractional,0.12', 'Fractional,0.26', 'Fractional,0.22', 'Fractional,0.23', 'Fractional,0.06', 'Fractional,0.171136', 'Fractional,0.17', 'Fractional,0.3', 'EPSS,16', 'EPSS,23', 'EPSS,11', 'EPSS,20', 'EPSS,19', 'EPSS,12.733', 'EPSS,23.6', 'EPSS,15.8968', 'EPSS,24.6', 'EPSS,19.4', 'EPSS,9.4', 'EPSS,40', 'EPSS,7.6', 'LVDD,5.8', 'LVDD,5

Union Set:  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}
final_union:  [{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,

['Age,71', 'Age,72', 'Age,55', 'Age,60', 'Age,57', 'Age,62', 'Age,46', 'Age,54', 'Age,73', 'Age,63', 'Age,61', 'Age,79', 'Age,58', 'Age,51', 'Age,70', 'Age,56', 'Age,59', 'Age,74', 'Age,66', 'Fractional,0.38', 'Fractional,0.26', 'Fractional,0.253', 'Fractional,0.16', 'Fractional,0.23', 'Fractional,0.33', 'Fractional,0.34', 'Fractional,0.14', 'Fractional,0.45', 'Fractional,0.15', 'Fractional,0.25', 'Fractional,0.09', 'Fractional,0.21', 'Fractional,0.241021', 'Fractional,0.51', 'Fractional,0.35', 'Fractional,0.3', 'Fractional,0.17', 'Fractional,0.187', 'Fractional,0.12', 'Fractional,0.2', 'Fractional,0.07', 'EPSS,9', 'EPSS,6', 'EPSS,4', 'EPSS,12.062', 'EPSS,8', 'EPSS,0', 'EPSS,13', 'EPSS,10', 'EPSS,4.2', 'EPSS,13.1', 'EPSS,11.0272', 'EPSS,9.3', 'EPSS,21.3', 'EPSS,9.4', 'EPSS,18.6', 'EPSS,11.9', 'EPSS,12', 'EPSS,8.8', 'EPSS,11', 'EPSS,7.1', 'EPSS,16.8', 'EPSS,9.1', 'EPSS,9.7', 'LVDD,4.6', 'LVDD,4.1', 'LVDD,5.75', 'LVDD,4.31', 'LVDD,5.43', 'LVDD,5.25', 'LVDD,4.49', 'LVDD,4', 'LVDD,3.73', '

['Wall-index,2.18']
final_union:  [{1, 4, 6, 7, 8, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 50, 51, 52, 54, 57, 58, 59, 60, 61, 64, 66, 67, 68, 70, 73, 74}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48

['Age,71', 'Age,72', 'Age,55', 'Age,60', 'Age,57', 'Age,62', 'Age,46', 'Age,54', 'Age,73', 'Age,63', 'Age,61', 'Age,79', 'Age,58', 'Age,51', 'Age,70', 'Age,56', 'Age,59', 'Age,74', 'Age,66', 'LVDD,4.6', 'LVDD,4.1', 'LVDD,5.75', 'LVDD,4.31', 'LVDD,5.43', 'LVDD,5.25', 'LVDD,4.49', 'LVDD,4']
['Age,71', 'Age,72', 'Age,55', 'Age,60', 'Age,57', 'Age,62', 'Age,46', 'Age,54', 'Age,73', 'Age,63', 'Age,61', 'Age,79', 'Age,58', 'Age,51', 'Age,70', 'Age,56', 'Age,59', 'Age,74', 'Age,66', 'LVDD,4.6', 'LVDD,4.1', 'LVDD,5.75', 'LVDD,4.31', 'LVDD,5.43', 'LVDD,5.25', 'LVDD,4.49', 'LVDD,4', 'LVDD,3.73']
['Age,71', 'Age,72', 'Age,55', 'Age,60', 'Age,57', 'Age,62', 'Age,46', 'Age,54', 'Age,73', 'Age,63', 'Age,61', 'Age,79', 'Age,58', 'Age,51', 'Age,70', 'Age,56', 'Age,59', 'Age,74', 'Age,66', 'LVDD,4.6', 'LVDD,4.1', 'LVDD,5.75', 'LVDD,4.31', 'LVDD,5.43', 'LVDD,5.25', 'LVDD,4.49', 'LVDD,4', 'LVDD,3.73', 'LVDD,4.29']
['Age,71', 'Age,72', 'Age,55', 'Age,60', 'Age,57', 'Age,62', 'Age,46', 'Age,54', 'Age,73', 

When values are list
['Pericardial,0']
['Pericardial,0', 'Pericardial,1']
Union Set:  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}
When value is single
['Age,63', 'Fractional,0.3']
When values are list
['Pericardial,0', 'Pericardial,1', 'EPSS,16']
['Pericardial,0', 'Pericardial,1', 'EPSS,16', 'EPSS,23']
['Pericardial,0', 'Pericardial,1', 'EPSS,16', 'EPSS,23', 'EPSS,11']
['Pericardial,0', 'Pericardial,1', 'EPSS,16', 'EPSS,23', 'EPSS,11', 'EPSS,20']
['Pericardial,0', 'Pericardial,1', 'EPSS,16', 'EPSS,23', 'EPSS,11', 'EPSS,20', 'EPSS,19']
['Pericardial,0', 'Pericardial,1', 'EPSS,16', 'EPSS,23', 'EPSS,11', 'EPSS,20', 'EPSS,19', 'EPSS,12.733']
['Pericardial,0', 'Pericardial,1', 'EPSS,16', 'EPSS,23', 'EPSS,11', 'EPSS,20', 'EPSS,19', 'EPSS,12.733', 'EPSS

In [43]:
dic

{'K_1': {1, 42},
 'K_2': {2, 42},
 'K_3': {3, 6, 32, 42, 45, 51, 62},
 'K_4': {4, 32, 42, 45, 51},
 'K_5': {5, 42, 44, 51, 72},
 'K_6': {3, 6, 42, 44, 63, 68, 72},
 'K_7': {7, 42, 44},
 'K_8': {8, 42},
 'K_9': {9, 42},
 'K_10': {10, 42},
 'K_11': {11, 52, 60},
 'K_12': {12},
 'K_13': {13, 42},
 'K_14': {14, 26, 42},
 'K_15': {15},
 'K_16': {16},
 'K_17': {17, 52, 66},
 'K_18': {18, 52},
 'K_19': {19, 26, 33, 42, 44, 51, 55, 62, 63, 68, 72},
 'K_20': {20},
 'K_21': {21, 52, 64},
 'K_22': {22, 38},
 'K_23': {23},
 'K_24': {24, 52, 60, 64, 67},
 'K_25': {25},
 'K_26': {14, 19, 26, 42, 43, 45},
 'K_27': {27, 42},
 'K_28': {28, 33, 55},
 'K_29': {29, 52},
 'K_30': {30, 33, 42, 51, 55, 62, 68},
 'K_31': {31, 42},
 'K_32': {3, 4, 32, 42, 44, 63, 72},
 'K_33': {19, 28, 30, 33, 42, 44},
 'K_34': {34, 42, 62},
 'K_35': {35, 42},
 'K_36': {36, 42, 62},
 'K_37': {37},
 'K_38': {22, 38, 39, 52, 57, 60, 64, 66, 67, 74},
 'K_39': {38, 39},
 'K_40': {40, 42, 51},
 'K_41': {41, 52},
 'K_42': {1,
  2,
 

In [44]:
def lowerApproximation(charac_set,concept):
    #set to contain lower approximations
    lower = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
  
        if set_value.issubset(set(concept)):
            lower = lower.union(set_value)
          
    return lower

In [45]:
lower_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    lower_approximations[str(item)] = lowerApproximation(dic,item)

In [46]:
lower_goal_list=list(lower_approximations.values())
print(lower_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 58, 59, 62, 63, 65, 68, 69, 71, 72}, {64, 66, 67, 70, 73, 74, 11, 15, 17, 18, 20, 21, 22, 23, 24, 29, 37, 38, 39, 41, 52, 57, 60, 61}]


In [47]:
def upperApproximation(charac_set,concept):
    #set to contain lower approximations
    upper = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
        upper = upper.union(set_value)
          
    return upper

In [48]:
upper_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    upper_approximations[str(item)] = upperApproximation(dic,item)

In [49]:
upper_goal_list=list(upper_approximations.values())
print(upper_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74}, {64, 66, 67, 70, 73, 74, 11, 15, 17, 18, 20, 21, 22, 23, 24, 29, 37, 38, 39, 41, 52, 57, 60, 61}]


In [50]:
first_column = list(dic.keys())

In [51]:
second_column = list(dic.values())

In [52]:
prob_approx = pd.DataFrame(
    {'charset_name': first_column,
     'charset_value': second_column
    })

In [53]:
def probabilisticApproximation(concept):
    prob = []
    for index, row in prob_approx.iterrows():
        if len(row['charset_value']) != 0:
            probability_conditional = len(row['charset_value'].intersection(set(concept))) / len(row['charset_value'])
        else:
            probability_conditional = 0.0
            
        prob.append(round(probability_conditional,2))
    return prob

In [54]:
def findMiddleApprox(concept):
    probapprox = set()
    cond_prob = probabilisticApproximation(concept) #Need to put the goal
    prob_approx['cond_probability'] = cond_prob
    print(prob_approx)
    for index, row in prob_approx.iterrows():
        part1,part2 = row['charset_name'].split("_")
       
        if row['cond_probability'] >= 0.50:
            if int(part2) in concept:
#                 print(row['charset_value'])
                probapprox = probapprox.union(row['charset_value'])
    
    return probapprox  

In [55]:
middle_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    middle_approximations[str(item)] = findMiddleApprox(item)

   charset_name                                      charset_value  \
0           K_1                                            {1, 42}   
1           K_2                                            {2, 42}   
2           K_3                         {32, 3, 6, 42, 45, 51, 62}   
3           K_4                                {32, 4, 42, 45, 51}   
4           K_5                                {5, 72, 42, 44, 51}   
5           K_6                         {3, 68, 6, 72, 42, 44, 63}   
6           K_7                                        {42, 44, 7}   
7           K_8                                            {8, 42}   
8           K_9                                            {9, 42}   
9          K_10                                           {10, 42}   
10         K_11                                       {11, 52, 60}   
11         K_12                                               {12}   
12         K_13                                           {42, 13}   
13         K_14     

In [56]:
middle_goal_list=list(middle_approximations.values())
print(middle_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74}, {64, 66, 67, 70, 73, 74, 11, 15, 17, 18, 20, 21, 22, 23, 24, 29, 37, 38, 39, 41, 52, 57, 60, 61}]


In [57]:
df3=df2

In [58]:
import pylab as pl

In [59]:
def findGoalIntersect(goal):
    goalIntersect = []
    
    for index, row in df3.iterrows():
        #List containing intersection of (a,v) pairs and goal
        goalIntersect.append(set(row['att_val']).intersection(set(goal)))
          
    #Check if goal_intersect column exists
    if 'goal_intersect' in df3:
        df3['goal_intersect'] = goalIntersect
    else:
        #Insert new column with the recent iteration
        df3.insert(2, 'goal_intersect', goalIntersect)

In [60]:
def updateGoalIntersect(goal):
    for index, row in df3.iterrows():
        if row['goal_intersect'] != set():
            row['goal_intersetct'] = set(row['att_val']).intersection(set(goal))

In [61]:
def findCases(df3):
    case_to_be = []
    #Find the cases with maximum goal coverage
    m = max(df3['goal_intersect'], key=len)
    possible_cases = [i for i, j in enumerate(df3['goal_intersect'].tolist()) if j == m]
    
    #Index of the case covering max goal and having min no. of elements
    new_df = df3.iloc[possible_cases,:]
    
    m1 = min(new_df['att_val'], key=len)
    
    for index,row in new_df.iterrows():
        if row['att_val'] == m1:
            case_to_be.append(index)
   
    return case_to_be[0]

In [62]:
def combineInterval(test_condition):
    
    test_num = [] #This will contain the conditions having interval
    test_str = [] #This will contain the conditions having no interval
    
    #Loop through to seprate contions having intervals and no intervals
    for item in test_condition:
        if ".." in item:
            test_num.append(item)
        else:
            test_str.append(item)
   
    #Group the conditions having interval based on same attributes
    grouped = [list(g) for k, g in groupby(test_num, lambda s: s.partition(',')[0])]
    
    final_list = []
    
    #Actually combining the intervals
    for list1 in grouped:
        greatest = 0
        smallest = 0
        for item in list1:
            part1,part2 = item.split(",")
            start,stop = part2.split("..")
            start = float(start)
            stop = float(stop)

            if greatest == 0 and smallest == 0:
                greatest = start
                smallest = stop

            if start > greatest:
                greatest = float(start)

            if stop < smallest:
                smallest = float(stop)

        con_tmp = part1 + "," + str(greatest) + ".." + str(smallest)
        final_list.append(con_tmp)  
            
    actual_condition = final_list + test_str
    
    return actual_condition

In [63]:
def dropCondition(condition,current_goal):
    
    for item in range(0,len(condition)):
        temp_att_val = []
        temp_cond = condition.copy() #use list.copy() as equal operator simply copies over the reference
        temp_cond.remove(condition[item])
       
        #temp_cond contains the elements after removing the current element
        for i in temp_cond:
            if i is not None:
                location = df3.index[df3['Cases'] == i].tolist()
                element = df3['att_val'].loc[location[0]]
                temp_att_val.append(set(element))
          
        #temp_att_val contains the actual value set of the corresponding cases
        #Find the intersection if the set has more than one element, otherwise no need
        if len(temp_att_val) > 1:
            intersection = set.intersection(*temp_att_val)
        
            #if the set still remains a subset of the original goal after removing current element
            #then set the current element to None as we want to drop this later
            if intersection.issubset(current_goal):
                condition[item] = None
    
    condition = [x for x in condition if x is not None]
   
    return condition   

In [64]:
def stepAlgo(df3,selected_case,current_goal,B,condition,concept_curr):
    
    rule_set = []
    original_goal = current_goal.copy()
    
    
    while current_goal != set():
        #Check if the selected case is a subset of the current goal
        #List of current case
        A = df3['att_val'].loc[selected_case] 

        if B == set():
            #Copy over the current set elements to B
            for i in range(len(A)):
                B.add(A[i])

        #Elements of intersection of current and previous set
        A = set(A).intersection(B)
        B = A.copy()
        
        #Check if intersecting elements are subset of Goal
        if A.issubset(original_goal):
            #Current goal is updated after discarding the already covered goal by new rule
            if len(A) != 0:
                current_goal = set(current_goal) - df3['goal_intersect'].loc[selected_case]
            else:
                current_goal = set()
            

            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the conditions of a Rule
            condition.append(curr_case)
            
            #Check for possibility of dropping conditions
            if len(condition) > 1:
                condition = dropCondition(condition,original_goal)
            
            #Combine the interval
            if len(condition) > 1:
                condition = combineInterval(condition)
            

            #Join conditions
            cond = ""
            for item in condition:
                cond = cond + "(" + str(item) + ")" + " & "

            cond = cond[:-2] + "->"
            rule = cond + " (" + concept + "," + str(concept_curr) + ")"
            rule_set.append(rule)
            
            
            #Reset everythng and continue for covering rest of the goal
            condition = []
            B = set()
            findGoalIntersect(current_goal)
            selected_case = findCases(df3)
            

        #If not a subset of current goal
        else:
            #Assign empty set for the selected case for next iteration
            df3['goal_intersect'].loc[selected_case] = set()
           
            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the case to the condition list
            condition.append(curr_case)
            
            #Check for Range overlapping of the remaining cases
            if ".." in curr_case:
                second_part = curr_case.split(',')[1]
                start = float(second_part.split('..')[0])
                end = float(second_part.split('..')[1])
                for index, row in df3.iterrows():
                    if ".." in row['Cases']:
                        part2 = row['Cases'].split(',')[1]
                        start1 = float(part2.split('..')[0])
                        end1 = float(part2.split('..')[1])

                        #Assign blank set for cases with overlapping ranges
                        if set((pl.frange(start,end))).issubset(pl.frange(start1,end1)) == True:
                            if start == start1 and end <= end1:
                                row['goal_intersect'] = set()
            
            updateGoalIntersect(A.intersection(current_goal))
            selected_case = findCases(df3)
                        
    return rule_set

In [65]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(middle_goal_list)):
    findGoalIntersect(list(middle_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,middle_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    print("End of goal")
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: MatplotlibDeprecationWarning: numpy.arange


End of goal
End of goal


In [66]:
print("Time to run the algorithm (for Middle Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Echo/Rules/Echo_40_attcon_middle.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Middle Approximation):  107.871 Sec


['(Pericardial,0.0..0.5) & (Age,46.0..66.73) -> (Alive,0)', '(Age,69.5..85.0) & (Fractional,0.14..0.5) & (Pericardial,0.0..0.5) -> (Alive,0)', '(LVDD,6.17..6.7) -> (Alive,0)']
['(Fractional,0.23..0.5) & (LVDD,5.13..5.22) & (Age,69.5..64.0) & (Pericardial,0.5..1.0) -> (Alive,1)', '(Wall-index,1.0..2.14) & (Age,82.0..85.0) -> (Alive,1)', '(Fractional,0.14..0.5) & (LVDD,3.5..3.55) -> (Alive,1)', '(Age,48.5..85.0) & (Fractional,0.1..0.07) -> (Alive,1)']


In [67]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(lower_goal_list)):
    findGoalIntersect(list(lower_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,lower_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    print("End of goal")
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: MatplotlibDeprecationWarning: numpy.arange


End of goal
End of goal


In [68]:
print("Time to run the algorithm (for Lower Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Echo/Rules/Echo_40_attcon_lower.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Lower Approximation):  109.695 Sec


['(Wall-score,10.0..15.25) & (Wall-index,1.0..1.27) & (LVDD,4.46..6.7) -> (Alive,0)', '(LVDD,3.5..4.37) & (Wall-score,10.0..14.5) & (Wall-index,1.0..1.24) -> (Alive,0)', '(Fractional,0.36..0.5) -> (Alive,0)']
['(Fractional,0.23..0.5) & (LVDD,5.13..5.22) & (Age,69.5..64.0) & (Pericardial,0.5..1.0) -> (Alive,1)', '(Wall-index,1.0..2.14) & (Age,82.0..85.0) -> (Alive,1)', '(Fractional,0.14..0.5) & (LVDD,3.5..3.55) -> (Alive,1)', '(Age,48.5..85.0) & (Fractional,0.1..0.07) -> (Alive,1)']


In [69]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(upper_goal_list)):
    findGoalIntersect(list(upper_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,upper_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    print("End of goal")
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: MatplotlibDeprecationWarning: numpy.arange


End of goal
End of goal


In [70]:
print("Time to run the algorithm (for Upper Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Echo/Rules/Echo_40_attcon_upper.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Upper Approximation):  102.433 Sec


['(Pericardial,0.0..0.5) & (Age,46.0..66.73) -> (Alive,0)', '(Age,69.5..85.0) & (Fractional,0.14..0.5) & (Pericardial,0.0..0.5) -> (Alive,0)', '(LVDD,6.17..6.7) -> (Alive,0)']
['(Fractional,0.23..0.5) & (LVDD,5.13..5.22) & (Age,69.5..64.0) & (Pericardial,0.5..1.0) -> (Alive,1)', '(Wall-index,1.0..2.14) & (Age,82.0..85.0) -> (Alive,1)', '(Fractional,0.14..0.5) & (LVDD,3.5..3.55) -> (Alive,1)', '(Age,48.5..85.0) & (Fractional,0.1..0.07) -> (Alive,1)']
